# 股票系統介紹

網址：https://merry.ee.ncku.edu.tw/stock/  
帳號為 luffy 帳號或是成大學號，密碼為成功入口密碼

這是為了成功大學機器學習與生物資訊學課程，所開發的一套股票模擬系統。系統上有 2004 年到現在每天台股的股市資料，使用者需要透過這些原始資料，實作一個預測模型。系統每天會執行使用者的模型，並依據預測結果決定明天要怎麼投資股票。每個人的總資產會依照時間繪製成折線圖和其他人比較，登入後可以查詢購買和賣出等操作細節。使用本系統的流程大致上可以分成以下幾個步驟：

1. 讀取資料
2. 使用模型
3. 產生決策
4. 提交程式

以下說明各個步驟的細節：

## 1. 讀取資料

在 luffy 的 `/home/mlb/res/stock/twse/json/` 目錄中，存放著每天每支股票的編號、開牌價、收盤價、最高最低價、成交量等資料。格式如下：

In [ ]:
stock = {
    "0050": {
        "adj_close": "84.90",
        "close": "84.95",
        "high": "85.80",
        "low": "84.90",
        "open": "85.80",
        "volume": "20996200"
    },
    "0051" : {
        "adj_close": "32.50",
        "close": "32.50",
        "high": "33.20",
        "low": "32.46",
        "open": "33.20",
        "volume": "92075"
    },
    "0052" : {
        "adj_close": "54.30",
        "close": "54.50",
        "high": "54.90",
        "low": "54.50",
        "open": "54.90",
        "volume": "102150"
    }
}

print(stock['0050'])
print(stock['0050']['close'])

## 2. 使用模型

讀取股票資料後，建構模型的方式可以參考 [classifier 教學](../week03_classifier/classifier.ipynb)。在使用模型時，必須開發一支程式，能夠讀取指定的時間、股票的資料產生特徵(例如取欲預測日前一周的所有股票，並以開盤價、收盤價以及平均價做為特徵)，並將這些特徵輸入已經訓練完成的模型，取得預測結果。

## 3. 產生決策

本系統將投資決策存成一個 `json` 檔案，稱之為 Decision File。其內容包含要買哪一隻股票、要買多少錢以及要持有多久等資訊。學生必須開發一支程式，將預測模型的結果轉換成 Decision File。通常預測結果只包含股票的漲跌(如果使用分類模型)或是預期股價(如果使用迴歸模型)，除非建立多個模型來決定 Decision File 中每一個參數，否則像是要買多少錢以及要持有多久等等的參數，都需要使用者另外寫條件來判斷。Decision File 的格式如下：

In [ ]:
[ # 一個陣列，每個元素是一個物件
    {
        "code": "1214", # 股票代碼
        "type": "buy", # 兩種操作，買(buy)/放空(short)
        "open_price": 16.75, # 等於或小於這個價格才買，交易才會發生

        # 操作幾單位；每個人一開始有 1 單位的虛擬現金；隨著投資可用的現金會變化
        # 每天投資時，系統會計算每個人每一天有一單位的錢可以花，
        # 每天投資時，系統會將所有權重加起來換算成百分比
        # 假設只買兩支股票，1214 權重為 1，2351 權重為 3，計算後花費分別為 1/4 與 3/4 乘上當天可用的現金
        # 以這個例子為例，用 1/4 買 1214；用 3/4 放空 2351
        "weigth": 1, 

        # 以下三個條件其中一個成力就會賣出
        "close_high_price": 16.8, # 比這個價格高就賣
        "close_low_price": 16.3 # 比這個價格低也賣(停損點)
        "life": 5, # 如果以上兩個條件都沒發生，會在 5 天會以收盤價賣
    },
    {
        "code": "2351",
        "life": 3,
        "type": "short",
        "weigth": 3,
        "open_price": 27.3,
        "close_high_price": 27.5,
        "close_low_price": 25.8
    }
]

# 注意實際操作時，Decision File 不能包含註解
# 注意本程式區段不是 python 程式碼，不能在 jupyter notebook 內執行

注意某日沒有 Decision File 或是內容為空時，系統會自動將所有現金投資大盤(TAIEX, 0050)，鼓勵使用者多多投資。根據過往經驗，跟著大盤投資會比預測模型表現來得糟。

## 4. 提交程式

每個使用者在 luffy 的帳號底下都有一個 `stock` 資料夾，結構為：  

* `~/stock/bin` 程式目錄，唯讀
* `~/stock/commit` 主要輸出目錄，用來存放 Decision File
* `~/stock/output` 備用輸出目錄，用來存放暫存檔
  
將上述流程需要的程式放入 `~/stock/bin` 目錄下即完成提交，系統會在每天的特定時間去抓取程式。以下是一個範例流程，詳細的內容及名稱可任意更改，過程中所有的暫存檔案建議放在 `~/stock/output` 目錄：
  
1. **讀取資料** 開發一個 `load.py` 程式來準備資料，因為每天的日期不同，這隻程式通常會需要當天日期作為參數。 
2. **使用模型** 開發一個 `predict.py` 程式來使用模型預測當天的結果。
3. **產生決策** 開發一個 `make_decision.py` 程式，根據預測的結果以及額外條件，產生 Decision File。最終的 Decision File 必須放在 `~/stock/commit` 目錄，檔名格式為 `YYYY-MM-DD_YYYY_MM_DD.json`，日期是欲預測日，例如 2018-09-10 的 Decision File 位置應為 `~/stock/commit/2018-09-10_2018-09-10.json`。  
 
最後，`~/stock/bin` 目錄還必須有一個 `run.sh` 程式(檔名不能更改)。`run.sh` 為 [shell script](http://blog.jex.tw/blog/2013/06/08/shell-script/)，裡面必須包含整個預測股票的流程，當 `run.sh` 成功執行之後，`~/stock/commit/` 下應該產生當天的 Decision File。系統在執行 `run.sh` 時，會在後面加上兩個參數，代表預測開始與結束的日期，例如 `sh run.sh 2018-09-10 2018-09-10`。這兩個參數在 shell script 可以用 `$1`、`$2` 取得。以下為 `run.sh` 的範例：

In [ ]:
# if command is `sh run.sh 2018-09-10 2018-09-10`

# generate the data to predict
./load.py $1 5 ../output/data 

# predict with a trained model
./predict.py ../output/data ../output/pred

# make decision
./make_decision.py ../output/pred ../commit/$1_$2.json

# the output file MUST be ../commit/2018-09-10_2018-09-10.json
# note that the code of this cell is python, please do not run this cell

## 備註

* 系統實際運作時，會先將 `~/stock/bin/` 目錄複製至獨立的虛擬機，創建對應的 `../commit` 與 `../output` 空目錄，再執行其中的 `run.sh`，所以執行時無法存取原本 `~/stock/bin/` 以外的檔案。  
* 模型應該預先訓練好，放置在 `~/stock/bin/` 中直接使用。系統執行 `run.sh` 時超過一分鐘會自動中斷，所以如果 `run.sh` 包含模型訓練通常會超過時限，導致無法產生 Decision File。  
* 透過系統網頁 Test & Simulate 頁籤中的 Test 功能，可以測試自己的 `run.sh` 能不能順利產生 Decision File。
* 系統網頁 Test & Simulate 頁籤中的 Simulate 提供模擬測試，在 luffy 上可以看到暫存檔，方便除錯。但由於本次修課人數眾多，暫時關閉。